# <center>Segmenting and Clustering Neighborhoods in Toronto</center>
### <center>By</center> 
### <center>C. J. Nyabando</center> 

In [1]:
!conda install beautifulsoup4
!conda install lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h7b6447c_2         3.7 MB
    certifi-2019.9.11          |           py36_0         154 KB
    ca-certificates-2019.8.28  |                0         132 KB
    beautifulsoup4-4.8.0       |           py36_0         147 KB
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.0-py36_0     
    ca-certificates: 2019.5.15-1       --> 2019.8.28-0      
    certifi:         2019.6.16-py36_1  --> 2019.9.11-py36_0 
    openssl:         1.1.1d-h7b6447c_1 --> 1.1.1d-h7b6447c_2


openssl-1.1.1d       | 3.7 MB   

In [57]:
#import libraries 
from bs4 import BeautifulSoup # for scrapping data
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [58]:
#scrape data
!wget -q -O 'Toronto_neighborhoods.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#load data
with open('Toronto_neighborhoods.html') as html_file:
    TorontoSoup = BeautifulSoup(html_file, 'lxml')
    TorontoCodes = TorontoSoup.find('table', {"class":"wikitable sortable"})
html_file.close()


In [59]:
# define the dataframe columns
column_names = ['Postalcode','Borough','Neighborhood'] 
# instantiate the dataframe
Toronto = pd.DataFrame(columns=column_names)
Toronto

,Postalcode,Borough,Neighborhood


In [62]:
#import data into dataframe
tablerow = TorontoCodes.findAll('tr')
rows = []
for tr in tablerow:
    td = tr.find_all('td')
    row = [tr.text.rstrip('\n') for tr in td]
    rows.append(row)
Toronto = pd.DataFrame(rows, columns=column_names) 
Toronto_filtered = Toronto[Toronto.Borough!='Not assigned']
Toronto_filtered = Toronto_filtered.groupby(['Postalcode', 'Borough'])['Neighborhood'].agg(','.join).reset_index(name='Neighborhood')

In [63]:
# The code was removed by Watson Studio for sharing.

In [64]:
#Add geo cordinates
Toronto_filtered['Latitude'] =  geo['Latitude']
Toronto_filtered['Longitude'] =  geo['Longitude']
Toronto_filtered

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [65]:
# create map of Toronto Neighborhoods
#latitude = Toronto_filtered['Latitude'].astype(float)
#longitude = Toronto_filtered['Longitude'].astype(float)
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Toronto_filtered['Latitude'], Toronto_filtered['Longitude'], Toronto_filtered['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
